<a href="https://colab.research.google.com/github/coffees-codes/Machine-Learning/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install evaluation

     |████████████████████████████████| 52 kB 767 kB/s 
  Created wheel for evaluation: filename=evaluation-0.0.2-py3-none-any.whl size=2479 sha256=f60c64ba3e5f9e83e93e3aec5d5cd72232a28d3579fcc3318e83d583ab8e76a7
  Stored in directory: /root/.cache/pip/wheels/18/75/b8/63929bfb42b59346c83a70f51ef709888e344cb34172930ef2
  Created wheel for python-gflags: filename=python_gflags-3.1.2-py3-none-any.whl size=57386 sha256=1cd747a7ea4e69798a247aa8e3cc4f0f87f6f8b94cc8900a5149d4f7987f57de
  Stored in directory: /root/.cache/pip/wheels/df/27/8a/e46bf628958f821f7d2092f276f5a81e184bcf1a1ccdeecd95
Successfully built evaluation python-gflags


In [3]:
import pandas as pd
import random
import json

In [4]:
def convert_input(data):
  for entry in data:
    entry["text"]=entry["text"].replace("LOCATION2","location - 2").replace("LOCATION1","location - 1")
    for opinion in entry["opinions"]:
      opinion["target_entity"]=opinion["target_entity"].replace("LOCATION2","location - 2").replace("LOCATION1","location - 1")
      opinion["aspect"]=opinion["aspect"].replace("transit-location","transit location")
  return data

In [5]:
sentihood_location=["location - 1","location - 2"]
sentihood_aspects=["general","price","safety","transit location"]
sentihood_sentiments=["none","positive","negative"]

def generate_sentiments_NLI_B(data) :
  output=[]
  for entry in data:
    id = entry["id"]
    original_sentence=entry["text"]
    for location in sentihood_location:
      if location in original_sentence:
        for aspect in sentihood_aspects:
          current_output=[]
          for sentiment in sentihood_sentiments:
            auxilary_sentence = f"{location}-{aspect}-{sentiment}"
            current_output.append([id,original_sentence,auxilary_sentence,1 if sentiment == "none" else 0 ])
          for opinion in entry["opinions"]:
            if opinion["target_entity"]==location and opinion["aspect"]==aspect:
              current_output[0][-1]=0
              if opinion["sentiment"]=="Positive":
                current_output[1][-1]=1
              if opinion["sentiment"]=="Negative":
                current_output[2][-1]=1
          output.extend(current_output)
  loc1_rows=sorted([row for row in output if "location - 1" in row[2]],key = lambda el: el[0])
  loc2_rows=sorted([row for row in output if "location - 2" in row[2]],key = lambda el: el[0])
  output = [["id","original_sentence","auxilary_sentence","label"]]
  output += loc1_rows+loc2_rows
  df=pd.DataFrame(output)
  return df




In [6]:
with open("/content/jsonvalidator.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_B(data)
  df.to_csv("auxilary_NLIB.csv",index=False,header=False)

FileNotFoundError: ignored

In [7]:
auxdata=pd.read_csv(r"/content/auxilary_NLIB.csv",index_col=0)
auxdata.head()

FileNotFoundError: ignored

In [ ]:
with open("/content/jsonvalidator_test.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_B(data)
  df.to_csv("auxilary_NLIB_test.csv",index=False,header=False)
auxdata_test=pd.read_csv(r"/content/auxilary_NLIB_test.csv",index_col=0)
auxdata_test.head()

,original_sentence,auxilary_sentence,label
id,,,
0,If you want to go out drinking where the cool ...,location - 1-general-none,1
0,If you want to go out drinking where the cool ...,location - 1-general-positive,0
0,If you want to go out drinking where the cool ...,location - 1-general-negative,0
0,If you want to go out drinking where the cool ...,location - 1-price-none,1
0,If you want to go out drinking where the cool ...,location - 1-price-positive,0


In [ ]:
with open("/content/jsonvalidator_val.json","r") as f:
  data = json.loads(f.read())
  data = convert_input(data)
  df=generate_sentiments_NLI_B(data)
  df.to_csv("auxilary_NLIB_val.csv",index=False,header=False)
auxdata_val=pd.read_csv(r"/content/auxilary_NLIB_val.csv",index_col=0)
auxdata_val.head()


,original_sentence,auxilary_sentence,label
id,,,
0,I stayed in location - 1 and loved the area,location - 1-general-none,0
0,I stayed in location - 1 and loved the area,location - 1-general-positive,1
0,I stayed in location - 1 and loved the area,location - 1-general-negative,0
0,I stayed in location - 1 and loved the area,location - 1-price-none,1
0,I stayed in location - 1 and loved the area,location - 1-price-positive,0


In [ ]:
train_original_sentences = []
train_auxiliary_sentences = []
train_labels = []
for row in auxdata:
        train_original_sentences.append(row[1])
        train_auxiliary_sentences.append(row[2])
        train_labels.append(row[3])
test_original_sentences = []
test_auxiliary_sentences = []
test_labels = []
for row in auxdata_test:
        test_original_sentences.append(row[1])
        test_auxiliary_sentences.append(row[2])
        test_labels.append(row[3]) 
val_original_sentences = []
val_auxiliary_sentences = []
val_labels = []
for row in auxdata_val:
        val_original_sentences.append(row[1])
        val_auxiliary_sentences.append(row[2])
        val_labels.append(row[3])                       

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_original_sentences, train_auxiliary_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_original_sentences,val_auxiliary_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_original_sentences,test_auxiliary_sentences, truncation=True, padding=True)

train_encodings


{'input_ids': [[101, 1054, 102, 1045, 102], [101, 1057, 102, 1060, 102], [101, 1037, 102, 1038, 102]], 'token_type_ids': [[0, 0, 0, 1, 1], [0, 0, 0, 1, 1], [0, 0, 0, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ABSA_Dataset(train_encodings, train_labels)
val_dataset=ABSA_Dataset(val_encodings, val_labels)
test_dataset=ABSA_Dataset(test_encodings, test_labels)

print(train_dataset)

In [ ]:
from scipy.special import softmax
import evaluation

def get_predictions(data, task, dataset_type):
    predicted_labels = []
    scores = []       
    count_aspect_rows = 0
    current_aspect_scores = []
    for row in data:
       current_aspect_scores.append(row[2])
       count_aspect_rows += 1
       if count_aspect_rows % 3 == 0:
          sum_current_aspect_scores = np.sum(current_aspect_scores)
          current_aspect_scores = [score / sum_current_aspect_scores for score in current_aspect_scores]
          scores.append(current_aspect_scores)
          predicted_labels.append(np.argmax(current_aspect_scores))
          current_aspect_scores = []
    return predicted_labels, scores

In [ ]:
def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        data = np.insert(scores, 0, predicted_labels, axis=1)
        predicted_labels, scores = get_predictions(data, task, dataset_type)
        test_labels = get_test_labels(f"{base_dir}/data", dataset_type)
        metrics = {}
        metrics["strict_acc"] = evaluation.compute_sentihood_aspect_strict_accuracy(test_labels, predicted_labels)
        metrics["F1"] = evaluation.compute_sentihood_aspect_macro_F1(test_labels, predicted_labels)
        metrics["aspect_AUC"] = evaluation.compute_sentihood_aspect_macro_AUC(test_labels, scores)
        sentiment_macro_AUC, sentiment_accuracy = evaluation.compute_sentihood_sentiment_classification_metrics(test_labels, scores)
        metrics["sentiment_acc"] = sentiment_accuracy
        metrics["sentiment_AUC"] = sentiment_macro_AUC
        return metrics

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

from transformers import logging
logging.set_verbosity_debug()


epochs = 4
batch_size = 24
num_steps = len(train_dataset) * epochs // batch_size
warmup_steps = num_steps // 10  # 10% of the training steps
save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


training_args = TrainingArguments(
    output_dir = r"C:\Users\Saurabh\OneDrive - Manipal Academy of Higher Education\Desktop\MRM-Taskphase",          
    num_train_epochs = epochs,              
    per_device_train_batch_size = batch_size,  
    per_device_eval_batch_size = batch_size,   
    warmup_steps = warmup_steps,   
    weight_decay = 0.01,               
    logging_dir = r"C:\Users\Saurabh\OneDrive - Manipal Academy of Higher Education\Desktop\MRM-Taskphase",            
    logging_steps = 10,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    save_steps = save_steps
)

config = BertConfig.from_pretrained(
    'bert-base-uncased',
    architectures = ['BertForSequenceClassification'],
    hidden_size = 768,
    num_hidden_layers = 12,
    num_attention_heads = 12,
    hidden_dropout_prob = 0.1,
    num_labels = 2
)    

load_finetuned_model = False
if not load_finetuned_model:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics                     
    )
    trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_h

TypeError: ignored

In [ ]:
evaluation_result = trainer.evaluate(t_dataset)
print(evaluation_result)
